In [46]:
import os
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


import data 

In [47]:
directory="C:/Users/chenl/market data export/"
file="QQQ_data.csv"
path=directory+file
QQQ=pd.read_csv(path,parse_dates=["Unnamed: 0"])
QQQ.rename(columns={"Unnamed: 0":"Date"},inplace=True)
QQQ.set_index("Date",inplace=True)
QQQ.index=pd.to_datetime(QQQ.index)

#other index etf
directory="C:/Users/chenl/market data export/"
file_SPY="SPY_data.csv"
file_DIA="DIA_data.csv"
file_IWM="IWM_data.csv"

path=directory+file_SPY
SPY=pd.read_csv(path,parse_dates=["Unnamed: 0"])
SPY.rename(columns={"Unnamed: 0":"Date"},inplace=True)
SPY.set_index("Date",inplace=True)
SPY.index=pd.to_datetime(SPY.index)

path=directory+file_DIA
DIA=pd.read_csv(path,parse_dates=["Unnamed: 0"])
DIA.rename(columns={"Unnamed: 0":"Date"},inplace=True)
DIA.set_index("Date",inplace=True)
DIA.index=pd.to_datetime(DIA.index)

path=directory+file_IWM
IWM=pd.read_csv(path,parse_dates=["Unnamed: 0"])
IWM.rename(columns={"Unnamed: 0":"Date"},inplace=True)
IWM.set_index("Date",inplace=True)
IWM.index=pd.to_datetime(IWM.index)

# join table to QQQ
QQQ = pd.merge(QQQ, SPY[['Close']], how='left', left_index=True, right_index=True, suffixes=('', '_SPY'))
QQQ = pd.merge(QQQ, DIA[['Close']], how='left', left_index=True, right_index=True, suffixes=('', '_DIA'))
QQQ = pd.merge(QQQ, IWM[['Close']], how='left', left_index=True, right_index=True, suffixes=('', '_IWM'))


In [48]:
# create daily return
QQQ["QQQ 1d movement"]=QQQ["Close"].pct_change()
QQQ["SPY 1d movement"]=QQQ["Close_SPY"].pct_change()
QQQ["DIA 1d movement"]=QQQ["Close_DIA"].pct_change()
QQQ["IWM 1d movement"]=QQQ["Close_IWM"].pct_change()

In [49]:
# Basic eda
# QQQ.head()
# QQQ.isnull().sum()
# QQQ.info()
QQQ.describe()
# QQQ.index.dtype
# QQQ["Close"].plot(figsize=(20,7))
# plt.title("close price",fontsize=16)
# plt.xlabel("Date",fontsize=16)
# plt.ylabel("close price",fontsize=16)
# plt.show()

,Open,High,Low,Close,Adj Close,Volume,Close_SPY,Close_DIA,Close_IWM,QQQ 1d movement,SPY 1d movement,DIA 1d movement,IWM 1d movement
count,2264.000000,2264.000000,2264.000000,2264.000000,2264.000000,2.264000e+03,2264.000000,2264.000000,2264.000000,2263.000000,2263.000000,2263.000000,2263.000000
mean,219.456802,221.117036,217.674267,219.520247,214.199603,4.184231e+07,312.067867,262.921188,159.250861,0.000709,0.000437,0.000397,0.000334
std,95.366049,96.209859,94.465896,95.396082,96.492964,2.275478e+07,86.743290,62.596477,33.670544,0.013998,0.011449,0.011373,0.014322
min,94.230003,97.050003,84.739998,96.320000,90.576317,7.079300e+06,182.860001,156.490005,94.790001,-0.119788,-0.109424,-0.127612,-0.132669
25%,131.887497,132.297501,131.422504,131.779999,125.509834,2.517672e+07,236.462502,206.907505,134.880001,-0.005137,-0.003947,-0.004030,-0.006852
50%,186.199997,187.595001,185.000000,186.120003,180.500938,3.636235e+07,288.870010,260.059998,155.000000,0.001135,0.000531,0.000653,0.000768
75%,308.477501,311.042496,305.264992,308.935013,304.310631,5.320265e+07,396.342491,328.482506,180.825005,0.007769,0.005920,0.005441,0.007930
max,412.670013,412.920013,410.940002,411.500000,411.500000,1.986858e+08,477.709991,377.029999,242.559998,0.084706,0.090603,0.110192,0.091491


MA create

In [50]:
ma4=4
ma5=5
ma9=9
ma14=14
ma18=18
ma20=20
ma50=50
ma100=100
ma200=200
QQQ["9ma"]=QQQ["Close"].rolling(window=ma9,min_periods=1).mean()
QQQ["20ma"]=QQQ["Close"].rolling(window=ma20,min_periods=1).mean()
QQQ["50ma"]=QQQ["Close"].rolling(window=ma50,min_periods=1).mean()
QQQ["100ma"]=QQQ["Close"].rolling(window=ma100,min_periods=1).mean()
QQQ["200ma"]=QQQ["Close"].rolling(window=ma200,min_periods=1).mean()

#volumne adjust EMA
# QQQ['EMA_20'] = QQQ['Close'].ewm(span=20, adjust=False).mean()

In [51]:
# 50/200 cross & triple cross

In [52]:
QQQ['cross'] = 'No Cross'
startday = QQQ.index[0] + pd.Timedelta(days=60)
# Find where the golden cross occurs
golden_cross_indices = (QQQ['50ma'] > QQQ['200ma']) & (QQQ['50ma'].shift(1) <= QQQ['200ma'].shift(1))#create a series of True/false, bool with date as index
QQQ.loc[golden_cross_indices&(QQQ.index>=startday), 'cross'] = 'Golden Cross'#merge the series with dataset

# Find where the death cross occurs
death_cross_indices = (QQQ['50ma'] < QQQ['200ma']) & (QQQ['50ma'].shift(1) >= QQQ['200ma'].shift(1))
QQQ.loc[death_cross_indices&(QQQ.index>=startday),'cross'] = 'Death Cross'

#cross signals 9-20-50
QQQ['triple cross'] = 'No Cross'
# Find where the cross occurs, 9ma always cross 20am before cross 50ma
triplecross_indices = (
    (QQQ['9ma'] > QQQ['50ma']) 
    &(QQQ['9ma'] > QQQ['20ma']) 
    &((QQQ["20ma"]>QQQ["50ma"])& (QQQ['20ma'].shift(1) <= QQQ['50ma'].shift(1))) 
)               
#apply after 50 days of starting point
startday = QQQ.index[0] + pd.Timedelta(days=50)
QQQ.loc[triplecross_indices & (QQQ.index>=startday), 'triple cross'] = 'triple cross'#merge the series with dataset


In [53]:
# need convert cross or no cross into 1/0

bollinger band: upper and lower bound with 2 sigma, current price away from moving average (in sigma)

In [54]:
QQQ['Middle_BB'] = QQQ['Close'].rolling(window=20).mean()
QQQ['Upper_BB'] = QQQ['Middle_BB'] + 2 * QQQ['Close'].rolling(window=20).std()
QQQ['Lower_BB'] = QQQ['Middle_BB'] - 2 * QQQ['Close'].rolling(window=20).std()
QQQ['Distance_from_Middle'] = (QQQ['Close'] - QQQ['Middle_BB']) / (QQQ['Close'].rolling(window=20).std())
QQQ["Above_upperbb"]=np.where(QQQ['Close'] > QQQ['Upper_BB'], 1, 0)
QQQ["Below_lowerbb"]=np.where(QQQ['Close'] < QQQ['Lower_BB'], 1, 0)

RSI: run 14day and 21day RSI, and test if current is above 70 & below 30

In [55]:
delta = QQQ['Close'].diff()#absolute result
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain_14d = gain.rolling(window=14).mean()
avg_loss_14d = loss.rolling(window=14).mean()

avg_gain_21d = gain.rolling(window=21).mean()
avg_loss_21d = loss.rolling(window=21).mean()

rs_14d = avg_gain_14d/ avg_loss_14d 
rs_21d = avg_gain_21d / avg_loss_21d

QQQ['RSI_14d'] = 100 - (100 / (1 + rs_14d))
QQQ['RSI_21d'] = 100 - (100 / (1 + rs_21d))

QQQ["RSI_Above_70"]=np.where(QQQ['RSI_21d']>70,1,0)
QQQ["RSI_Below_30"]=np.where(QQQ['RSI_21d']<30,1,0)

MACD: generate crossing signal of MACD and Signal line, commonly use 26,12, 9 days ewm, can use longer if needed

In [56]:
#generate line
QQQ['MACD'] = QQQ['Close'].ewm(span=12, adjust=False).mean() - QQQ['Close'].ewm(span=26, adjust=False).mean()
QQQ['Signal_Line'] = QQQ['MACD'].ewm(span=9, adjust=False).mean()
#signals buy and sell
QQQ["MACD_cross_above"]=np.where((QQQ['MACD']>QQQ['Signal_Line']) & (QQQ['MACD'].shift(1) < QQQ['Signal_Line'].shift(1)),1,0 )
QQQ["MACD_cross_below"]=np.where((QQQ['MACD']<QQQ['Signal_Line']) & (QQQ['MACD'].shift(1) > QQQ['Signal_Line'].shift(1)),1,0 )

price change divergent:different in price movement %

In [57]:
QQQ["1m_return_diff_QQQ_IWM"]=QQQ['Close'].pct_change(periods=30) * 100-IWM['Close'].pct_change(periods=30) * 100
QQQ["1m_return_diff_QQQ_DIA"]=QQQ['Close'].pct_change(periods=30) * 100-DIA['Close'].pct_change(periods=30) * 100


# target variables - select one to fit model and predict

In [58]:
#price direction 1 stand for positive movement, -1 for negative movement
#1mon
QQQ['Future1M_movement_direction'] = np.where((QQQ['Close'].shift(-30) >= QQQ['Close']),1,-1)
#2mon
QQQ['Future2M_movement_direction'] = np.where((QQQ['Close'].shift(-60) >= QQQ['Close']),1,-1)
#15d
QQQ['Future15d_movement_direction'] = np.where((QQQ['Close'].shift(-15) >= QQQ['Close']),1,-1)

#price change 1m
QQQ['Future1M_movement'] = (QQQ['Close'].shift(-30)/QQQ["Close"]-1 )* 100 
#price change 2m
QQQ['Future2M_movement'] = (QQQ['Close'].shift(-60)/QQQ["Close"]-1 )* 100 
#price change 15days
QQQ['Future15d_movement'] = (QQQ['Close'].shift(-15)/QQQ["Close"]-1 )* 100 




delete some NA due

In [60]:
QQQ.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Close_SPY',
       'Close_DIA', 'Close_IWM', 'QQQ 1d movement', 'SPY 1d movement',
       'DIA 1d movement', 'IWM 1d movement', '9ma', '20ma', '50ma', '100ma',
       '200ma', 'cross', 'triple cross', 'Middle_BB', 'Upper_BB', 'Lower_BB',
       'Distance_from_Middle', 'Above_upperbb', 'Below_lowerbb', 'RSI_14d',
       'RSI_21d', 'RSI_Above_70', 'RSI_Below_30', 'MACD', 'Signal_Line',
       'MACD_cross_above', 'MACD_cross_below', '1m_return_diff_QQQ_IWM',
       '1m_return_diff_QQQ_DIA', 'Future1M_movement_direction',
       'Future2M_movement_direction', 'Future15d_movement_direction',
       'Future1M_movement', 'Future2M_movement', 'Future15d_movement'],
      dtype='object')